In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/tratamiento-correcto-columnas'

/content/drive/My Drive/Colab Notebooks/tratamiento-correcto-columnas


In [ ]:
import os
import random as rd
import numpy as np
import pandas as pd

In [ ]:
ruta_iniciales = os.path.join(os.getcwd(), 'archivos-iniciales')
ruta_resultados = os.path.join(os.getcwd(), 'archivos-resultados')

In [ ]:
archivos_iniciales = os.listdir(ruta_iniciales)
archivos_iniciales

['data-ct-arroz.csv', 'data-ct-maiz.csv']

**ARROZ**

### CRITERIO HECTAREA MAYOR

In [ ]:
ruta_arroz = os.path.join(ruta_iniciales, 'data-ct-arroz.csv')
arroz = pd.read_csv(ruta_arroz)
arroz

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,ec19_actafec,ec19_dadinsu,ec19_dconmob,ec19_dtraterre,ec19_dofretrab,ec19_pveprfalt,ec19_pveprausc,ec19_accecred,ec19_recayamag,ec19_recayusgr,ec19_recayugc,ec19_recayugadm,ec19_recayugadp,ec19_recayuong,ec19_recayuotro,ec19_recayudesc,eu_superficie_ha,eu_k1301,eu_k1302,eu_k1303,eu_k1304,eu_k1305,eu_k1306,eu_k1307,eu_k1308,eu_k1309,eu_k1310,eu_k1311,eu_k1312,eu_trs,eu_trc,eu_tro,eu_tn,...,ct_cantidad_fun_pq,ct_umed_fun_pq,ct_color_fun_pq,ct_cantidad_pq,ct_umed_pq,ct_color_pq,ct_trabocacont,ct_trabfami,ct_piensasem_int,ct_supersem_int,ct_supersemha_int,ct_messiem_int,ct_condcult_int,ct_codcultiv1_int,ct_codcultiv2_int,ct_nclavr,ct_trs,ct_trc,ct_tro,ct_tn,ct_prod,ct_superdida,superdida,ct_sequia,ct_helada,ct_plagas,ct_enfermedades,ct_inundacion,ct_otra,ct_secomun,ct_semejorada,ct_secertificada,ct_sehibrinac,ct_sehibrinter,Ventas,ct_sriego,su_fertilizada,su_plaguicidas,ct_nuevacondicion,fact_exp_fin_cultivo
0,9205002083060005,9,20,50,2,8306,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0000,2,2.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,50.0,5.0,4.0,NaN,NaN,NaN,10.0,0.0,1,60.0,60.0000,7.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,400.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0000,NaN,NaN,NaN,NaN,400.000000,60.0000,60.0000,60.0000,1,0.459974
1,9205002083060005,9,20,50,2,8306,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0000,2,2.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,45.0,5.0,4.0,NaN,NaN,NaN,10.0,0.0,1,90.0,90.0000,1.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,390.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0000,NaN,NaN,NaN,NaN,390.000000,90.0000,90.0000,90.0000,1,0.459974
2,9065202067080004,9,6,52,2,6708,4,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,8.4672,8,8.0,NaN,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1,12.0,8.4672,6.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,36.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4672,NaN,NaN,NaN,NaN,36.000000,8.4672,8.4672,8.4672,1,0.296543
3,12035002137630006,12,3,50,2,13763,6,1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,36.6900,10,10.0,NaN,3.0,3.0,NaN,NaN,NaN,NaN,7.0,7.0,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,1,2.0,2.0000,1.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,1.363636,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0000,NaN,NaN,NaN,NaN,1.363636,NaN,2.0000,2.0000,1,5.973195
4,9115402133430004,9,11,54,2,13343,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0000,2,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1,2.0,2.0000,12.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,7.636364,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0000,NaN,NaN,NaN,NaN,7.636364,2.0000,2.0000,2.0000,1,0.535500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,9055003002750002,9,5,50,3,275,2,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.5000,2,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0.5,0.5000,8.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,1.954545,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5000,NaN,NaN,NaN,NaN,1.954545,0.5000,0.5000,0.5000,1,191.125582
1519,9055003002750002,9,5,50,3,275,2,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.5000,2,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0.5,0.5000,2.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,0.977273,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5000,NaN,NaN,NaN,NaN,0.977273,0.5000,0.5000,0.5000,1,191.125582
1520,9065002059280003,9,6,50,2,5928,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0000,1,1.0,NaN,1.0,1.0,NaN,NaN

In [ ]:
arroz['Identificador'].duplicated().sum()

524

In [ ]:
arroz_res = arroz.drop_duplicates(subset=['Identificador'], keep='first', ignore_index=True)
arroz_res

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,ec19_actafec,ec19_dadinsu,ec19_dconmob,ec19_dtraterre,ec19_dofretrab,ec19_pveprfalt,ec19_pveprausc,ec19_accecred,ec19_recayamag,ec19_recayusgr,ec19_recayugc,ec19_recayugadm,ec19_recayugadp,ec19_recayuong,ec19_recayuotro,ec19_recayudesc,eu_superficie_ha,eu_k1301,eu_k1302,eu_k1303,eu_k1304,eu_k1305,eu_k1306,eu_k1307,eu_k1308,eu_k1309,eu_k1310,eu_k1311,eu_k1312,eu_trs,eu_trc,eu_tro,eu_tn,...,ct_cantidad_fun_pq,ct_umed_fun_pq,ct_color_fun_pq,ct_cantidad_pq,ct_umed_pq,ct_color_pq,ct_trabocacont,ct_trabfami,ct_piensasem_int,ct_supersem_int,ct_supersemha_int,ct_messiem_int,ct_condcult_int,ct_codcultiv1_int,ct_codcultiv2_int,ct_nclavr,ct_trs,ct_trc,ct_tro,ct_tn,ct_prod,ct_superdida,superdida,ct_sequia,ct_helada,ct_plagas,ct_enfermedades,ct_inundacion,ct_otra,ct_secomun,ct_semejorada,ct_secertificada,ct_sehibrinac,ct_sehibrinter,Ventas,ct_sriego,su_fertilizada,su_plaguicidas,ct_nuevacondicion,fact_exp_fin_cultivo
0,9205002083060005,9,20,50,2,8306,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0000,2,2.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,50.0,5.0,4.0,NaN,NaN,NaN,10.0,0.0,1,60.0,60.0000,7.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,400.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0000,NaN,NaN,NaN,NaN,400.000000,60.0000,60.0000,60.0000,1,0.459974
1,9065202067080004,9,6,52,2,6708,4,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,8.4672,8,8.0,NaN,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1,12.0,8.4672,6.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,36.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4672,NaN,NaN,NaN,NaN,36.000000,8.4672,8.4672,8.4672,1,0.296543
2,12035002137630006,12,3,50,2,13763,6,1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,36.6900,10,10.0,NaN,3.0,3.0,NaN,NaN,NaN,NaN,7.0,7.0,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,1,2.0,2.0000,1.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,1.363636,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0000,NaN,NaN,NaN,NaN,1.363636,NaN,2.0000,2.0000,1,5.973195
3,9115402133430004,9,11,54,2,13343,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0000,2,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1,2.0,2.0000,12.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,7.636364,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0000,NaN,NaN,NaN,NaN,7.636364,2.0000,2.0000,2.0000,1,0.535500
4,9115402135040002,9,11,54,2,13504,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404.0000,82,82.0,NaN,1.0,1.0,NaN,35.0,35.0,NaN,46.0,46.0,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,507.0,NaN,2.0,NaN,4.0,334.090909,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0000,NaN,NaN,NaN,NaN,334.090909,70.0000,70.0000,70.0000,1,2.068380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,9065402055150004,9,6,54,2,5515,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4112,11,10.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,10.0,10.0,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,1,2.0,1.4112,5.0,1.0,507.0,NaN,507.0,NaN,2.0,NaN,4.0,9.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4112,NaN,NaN,NaN,NaN,9.000000,1.4112,1.4112,1.4112,1,179.639679
995,9075003012980013,9,7,50,3,1298,13,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0000,2,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,507.0,NaN,2.0,NaN,4.0,1.500000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5000,NaN,NaN,NaN,NaN,1.500000,0.5000,0.5000,0.5000,1,262.677281
996,9055003002750002,9,5,50,3,275,2,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.5000,2,2.0,NaN,2.0,2.0,NaN,NaN,Na

In [ ]:
muestra = rd.sample(list(arroz['Identificador'][arroz['Identificador'].duplicated()]), 5)
muestra

[9135002039110011,
 9065302050430007,
 9065202068980008,
 9065202063020013,
 9065202065770005]

In [ ]:
def criterio_hectarea_mayor(df_repetidos, df_unicos, columna):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    veces = mascara.sum()
    if veces > 1 and df_repetidos[mascara][columna].nunique() > 1:
      array = df_repetidos[mascara]['ct_k509'].values
      array = np.nan_to_num(array, nan=-np.inf)
      pos_max = array.argmax()
      valor = df_repetidos[mascara][columna].values[pos_max]
      df_unicos.loc[id, columna] = valor
  df_unicos.reset_index(inplace=True)
  return df_unicos

In [ ]:
arroz_res = criterio_hectarea_mayor(arroz, arroz_res, 'ual_parr')
arroz_res = criterio_hectarea_mayor(arroz, arroz_res, 'ct_afecta_prod')
arroz_res = criterio_hectarea_mayor(arroz, arroz_res, 'ct_riego')

In [ ]:
# arroz[arroz['Identificador'].isin(muestra)][['Identificador', 'ual_parr', 'ct_k509', 'ct_afecta_prod', 'ct_riego', 'ct_numl', 'ct_numt']].sort_values('Identificador')
arroz[arroz['Identificador'] == 9205002084990003][['Identificador', 'ual_parr', 'ct_k509', 'ct_afecta_prod', 'ct_riego', 'ct_numl', 'ct_numt']]

,Identificador,ual_parr,ct_k509,ct_afecta_prod,ct_riego,ct_numl,ct_numt
350,9205002084990003,50,6.0,1.0,1,1,1
351,9205002084990003,50,6.0,1.0,1,2,2
352,9205002084990003,50,6.0,1.0,1,3,3


In [ ]:
# arroz_res[arroz_res['Identificador'].isin(muestra)][['Identificador', 'ual_parr', 'ct_k509', 'ct_afecta_prod', 'ct_riego', 'ct_numl', 'ct_numt']]
arroz_res[arroz_res['Identificador'] == 9205002084990003][['Identificador', 'ual_parr', 'ct_k509', 'ct_afecta_prod', 'ct_riego', 'ct_numl', 'ct_numt']]

,Identificador,ual_parr,ct_k509,ct_afecta_prod,ct_riego,ct_numl,ct_numt
227,9205002084990003,50,6.0,1.0,1,1,1


### CRITERIO SUMA

In [ ]:
def criterio_suma_valores(df_repetidos, df_unicos, columna):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    veces = mascara.sum()
    if veces > 1:
      array = df_repetidos[mascara][columna].values
      array = np.nan_to_num(array, nan=0)
      suma  = array.sum()
      df_unicos.loc[id, columna] = suma
  df_unicos.reset_index(inplace=True)
  return df_unicos

In [ ]:
arroz_res = criterio_suma_valores(arroz, arroz_res, 'Ventas')
arroz_res = criterio_suma_valores(arroz, arroz_res, 'ct_cantidad_npk_fq')

In [ ]:
arroz[arroz['Identificador'] == 9205002084990003][['Identificador', 'Ventas', 'ct_cantidad_npk_fq']]

,Identificador,Ventas,ct_cantidad_npk_fq
350,9205002084990003,112.5,5625.0
351,9205002084990003,107.5,5625.0
352,9205002084990003,64.5,3375.0


In [ ]:
arroz_res[arroz_res['Identificador'] == 9205002084990003][['Identificador', 'Ventas', 'ct_cantidad_npk_fq']]

,Identificador,Ventas,ct_cantidad_npk_fq
227,9205002084990003,284.5,14625.0


### CRITERIO BASTA QUE HAYA UN UNO PARA QUE TODOS SEAN UNO

In [ ]:
def criterio_suma_valores(df_repetidos, df_unicos, columna):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    veces = mascara.sum()
    if veces > 1:
      array = df_repetidos[mascara][columna].values
      array = np.nan_to_num(array, nan=0)
      booleano = np.any(array)
      valor = int(booleano)
      df_unicos.loc[id, columna] = valor
  df_unicos.reset_index(inplace=True)
  return df_unicos

In [ ]:
columnas = ['pa_arealm',	'pa_areempa',	'eq_earado',	'eq_semman',	'eq_emotoguadana',	'eq_ebmanual',	'eq_ebestaciona',	'eq_ecosgranfigr']
for columna in columnas:
  arroz_res = criterio_suma_valores(arroz, arroz_res, columna)

In [ ]:
arroz[arroz['Identificador'] == 9205002084990003][['Identificador'] + columnas]

,Identificador,pa_arealm,pa_areempa,eq_earado,eq_semman,eq_emotoguadana,eq_ebmanual,eq_ebestaciona,eq_ecosgranfigr
350,9205002084990003,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
351,9205002084990003,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
352,9205002084990003,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
arroz_res[arroz_res['Identificador'] == 9205002084990003][['Identificador'] + columnas]

,Identificador,pa_arealm,pa_areempa,eq_earado,eq_semman,eq_emotoguadana,eq_ebmanual,eq_ebestaciona,eq_ecosgranfigr
227,9205002084990003,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


<hr>

In [ ]:
arroz[arroz['Identificador'] == 9205002084990003][['Identificador', 'eu_superficie_ha']]

,Identificador,eu_superficie_ha
350,9205002084990003,65.0
351,9205002084990003,65.0
352,9205002084990003,65.0


In [ ]:
arroz_res[arroz_res['Identificador'] == 9205002084990003][['Identificador', 'eu_superficie_ha']]

,Identificador,eu_superficie_ha
227,9205002084990003,65.0


### GUARDAR CAMBIOS

In [ ]:
arroz_res['Identificador'] = arroz_res['Identificador'].astype('str')
arroz_res.to_csv(os.path.join(ruta_resultados, 'arroz.csv'), index=False, encoding='utf-8')

In [ ]:
arroz_res.to_stata(os.path.join(ruta_resultados, 'arroz.dta'), write_index=False)

In [32]:
'ec19_pveprausc' in arroz_res.columns

True